## 4. Tests of Normality

(1) Using the analogy principle, construct an estimator for the first k moments of the distribution of x. Use this to define a k-vector of moment restrictions $ g_{N}(\mu, \sigma) $ satisfying $ E g_{N}(\mu, \sigma) = 0  $ under the null hypothesis of normality.

In [39]:
import numpy as np
from math import factorial

def moment_conditions(mu, sigma, k, data):
    # list to store vector of moment restrictions
    moments = []
    # For m = 1, take the mean of the data:
    for m in range(1, k + 1):
        if m == 1: 
            moments.append(np.mean(data) - mu)
        elif m % 2 == 1: 
            # For subsequent odd moments:
            moments.append(np.mean((data - mu) ** m) - 0)
        else: 
            # For all even moments:
            moments.append(np.mean((data - mu) ** m) - sigma ** m * factorial(m - 1))
    return np.array(moments)

In [46]:
# set seed for reproducibility
np.random.seed(1)

data = np.random.normal(0, 1, 1000)  # create data from a normal distribution
k = 3  # Number of moment conditions
mu = 0  # true mean (for null hypothesis of normal distribution)
sigma = 1  # true standard deviation (for null hypothesis of normal distribution)

# Calculate moment conditions
moment_restriction_vector = moment_conditions(mu, sigma, k, data)

# Print moment restriction vector with labels
print("Moment Restriction Vector:")
for i, moment in enumerate(moment_restriction_vector):
    print(f"{i+1} moment:", moment)

Moment Restriction Vector:
1 moment: 0.03881247615960185
2 moment: -0.036124480902271716
3 moment: 0.047583678240587984


(2) What is the covariance matrix of the sample moment restrictions (again under the null)? I.e., what can be said about 
$ E g_{j}(\mu, \sigma) g_{j}(\mu, \sigma)⊤ $ - $ E g_{j}(\mu, \sigma) E g_{j}(\mu, \sigma)⊤ $?

The covariance matrix of the sample moment restrictions is a matrix of zeros. This is because the sample moment restrictions under the null hypothesis are expected to be constants. This means each moment restriction is independent of each other under the null hypothesis.

(3) Using your answers to the previous two questions, suggest a GMM-based test of the hypothesis of normality, taking k > 2.

$ H_{0} $: data is normally distributed

$ H_{1} $: data is not normally distributed

We use can use a GMM-based test to estimate mu and thetha by minimizing the moment conditions. Then, we can calculate a chi-square test statistic, comparing the GMM estimates and the moment restrictions to their expected values under the null hypothesis of normality. If our p-value is less than 0.05, we can reject our null hypothesis that the data is normally distributed.

(4) Implement the test you’ve devised using python. You may want to use scipy.stats.distributions.chi2.cdf and scipy.optimize.minimize.

In [47]:
from scipy.optimize import minimize
from scipy.stats import chi2
from scipy.stats import skew

# Define the sample moment restrictions (mean, variance, skewness)
def sample_moments(data):
    # length of data
    n = len(data)
    # calculate first 3 moments
    mean = np.mean(data)
    variance = np.var(data)
    skewness = skew(data)
    return np.array([mean, variance, skewness])

# Define the population moment restrictions for a normal distribution
def population_moments(params):
    mu, sigma = params
    return np.array([mu, sigma**2, 0])

# Define the objective function to be minimized
def objective(params, data):
    sample_moments_val = sample_moments(data)
    population_moments_val = population_moments(params)
    return np.sum((sample_moments_val - population_moments_val) ** 2)

# Perform GMM estimation to estimate parameters mu and sigma
def perform_gmm(data):
    true_values = [0, 1]
    # Minimize the objective function
    result = minimize(objective, true_values, args=(data,))
    return result.x

# Test of normality using GMM
def test_normality(data):
    # estimate parameters using GMM
    mu, sigma = perform_gmm(data)
    # calculate the test statistic with sample/population moments
    sample_moments_val = sample_moments(data)
    population_moments_val = population_moments([mu, sigma])
    test_statistic = np.sum((sample_moments_val - population_moments_val) ** 2)
    # Calculate the p-value using chi-squared distribution
    df = len(sample_moments_val)  # Degrees of freedom
    p_value = 1 - chi2.cdf(test_statistic, df)
    return test_statistic, p_value, sample_moments_val

test_statistic, p_value, sample_moments_val = test_normality(data)
print("Test Statistic:", test_statistic)
print("P-value:", p_value)
print("Sample Moments:", sample_moments_val)

Test Statistic: 0.004672034352744824
P-value: 0.9999151856735445
Sample Moments: [ 0.03881248  0.96236911 -0.06835228]


We find that the observed data appears to be drawn from a normal distribution.

(5) What can be said about the optimal choice of k?

The number of moment restrictions should be less than or equal to the number of parameters. If there are more moment restrictions than parameters, we end up with an overidentified model. 

(6) Compare the GMM estimates of (μ, σ) to the maximum likelihood estimates of these parameters. Do they differ? Why?

In [24]:
from scipy.stats import norm

# Find MLE of mu and sigma
mu_mle, sigma_mle = norm.fit(data)

# Print MLE estimates
print("MLE of mu:", mu_mle)
print("MLE of sigma:", sigma_mle)

MLE of mu: 0.0017620340003794831
MLE of sigma: 0.9996802473477188


No, they do not differ. Under the assumption of a normal distribution and with a large enough sample size, both GMM and MLE converge to the true values. 